In [31]:
import time
import odrive
from odrive.enums import *

print("Finding odrv...")
odrv0 = odrive.find_any()

def CALIB():
    print("starting calibration...")
    time.sleep(1)
    odrv0.axis0.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE
    odrv0.axis1.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE
    while odrv0.axis0.current_state != AXIS_STATE_IDLE:
        time.sleep(0.1)
    while odrv0.axis1.current_state != AXIS_STATE_IDLE:
        time.sleep(0.1)

    print("setting params axis 0...")
    odrv0.axis0.encoder.set_linear_count(0)
    odrv0.axis0.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL

    print("setting params axis 1...")
    odrv0.axis1.encoder.set_linear_count(0)
    odrv0.axis1.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL

CALIB()

Finding odrv...
starting calibration...
setting params axis 0...
setting params axis 1...


In [6]:
def STOP():
    odrv0.axis0.requested_state = AXIS_STATE_IDLE
    odrv0.axis1.requested_state = AXIS_STATE_IDLE
STOP()

In [32]:
def rezero(axis, dir): # 1 = CCW, -1 = CW <-- dir indicates which direction you wish it so spin in while in use
    if axis == 0:
        odrv0.axis0.controller.config.control_mode = CONTROL_MODE_POSITION_CONTROL
        odrv0.axis0.controller.config.input_mode = INPUT_MODE_PASSTHROUGH
    if axis == 1:
        odrv0.axis1.controller.config.control_mode = CONTROL_MODE_POSITION_CONTROL
        odrv0.axis1.controller.config.input_mode = INPUT_MODE_PASSTHROUGH

    if axis == 0:
        init = odrv0.axis0.controller.input_pos
        print(init)
        while dir * init > 0:
            odrv0.axis0.controller.input_pos = init
            init += -1 * dir * 0.01
            time.sleep(0.01)
    elif axis == 1:
        init = odrv0.axis1.controller.input_pos
        print(init)
        while dir * init > 0:
            odrv0.axis1.controller.input_pos = init
            init += -1 * dir * 0.01
            time.sleep(0.01)
rezero(axis=0, dir=-1)
rezero(axis=1, dir=1)

0.0
0.0


In [33]:
import math
L1 = 275
L2 = 275
def q1q2_from_xy(x, y):
    xD_mag = math.sqrt(x*x + y*y)
    c2 = (xD_mag * xD_mag - L1 * L1 - L2 * L2) / (2 * L1 * L2)
    if abs(c2) > 1:
        return None
    if c2 == 1:
        return math.atan2(y, x), 0, math.atan2(y, x), 0
    q2_1 = 1 * math.acos( c2 )
    q2_2 = -1 * math.acos( c2 )
    theta = math.atan2(y, x)
    q1_1 = theta - math.atan2(L2 * math.sin(q2_1), L1 + L2 * math.cos(q2_1))
    q1_2 = theta - math.atan2(L2 * math.sin(q2_2), L1 + L2 * math.cos(q2_2))
    return q1_1, q2_1, q1_2, q2_2

In [34]:
rezero(axis=0,dir=-1)
rezero(axis=1,dir=1)
time.sleep(1)

0.0
0.0


In [41]:
import time

time.sleep(1)

odrv0.axis0.controller.config.control_mode = CONTROL_MODE_POSITION_CONTROL
odrv0.axis0.controller.config.input_mode = INPUT_MODE_PASSTHROUGH
odrv0.axis1.controller.config.control_mode = CONTROL_MODE_POSITION_CONTROL
odrv0.axis1.controller.config.input_mode = INPUT_MODE_PASSTHROUGH

final_standing_height = 450

rezero(0, -1)
rezero(1, 1)

total_time = 0
N = 1
for i in range(N):
    max_q1 = -10000
    min_q1 = 10000
    max_q2 = -10000
    min_q2 = 10000
    t_ = 0
    time_start = time.time()
    while t_ < 2:
        q1_1, q2_1, q1_2, q2_2 = 0, 0, 0, 0
        if t_ < 1: # Down
            x = (final_standing_height - L1 - L2) * t_ + L1 + L2
            y = 0
            q1_1, q2_1, q1_2, q2_2 = q1q2_from_xy(x, y)
        elif t_ < 2: # Up
            x = (L1 + L2 - final_standing_height) * t_ + 2 * final_standing_height - L1 - L2
            y = 0
            q1_1, q2_1, q1_2, q2_2 = q1q2_from_xy(x, y)
        odrv0.axis1.controller.input_pos = q1_2 * (10 / (2 * math.pi))
        odrv0.axis0.controller.input_pos = q2_2 * (10 / (2 * math.pi))
        max_q1 = max(math.degrees(q1_2), max_q1)
        min_q1 = min(math.degrees(q1_2), min_q1)
        max_q2 = max(math.degrees(q2_2), max_q2)
        min_q2 = min(math.degrees(q2_2), min_q2)
        t_ += 0.0001
        time.sleep(0.0001)
    time_taken = time.time() - time_start
    print(f"Took: {time_taken} on itr {i}")
    total_time += time_taken
    time.sleep(0.1)

print(f"Average time taken per move: {total_time / N}")
print(f"Min q1: {min_q1}, max q1: {max_q1}")
print(f"Min q2: {min_q2}, max q2: {max_q2}")

-8.68469612669287e-07
4.342348063346435e-07
Took: 15.32090711593628 on itr 0
Average time taken per move: 15.32090711593628
Min q1: 0.0, max q1: 35.0968012275829
Min q2: -70.1936024551658, max q2: 0.0


In [30]:
import time

time.sleep(1)

final_standing_height = 500

total_time = 0
N = 10
t_ = 0
time_start = time.time()
while t_ < 2:
    q1_1, q2_1, q1_2, q2_2 = 0, 0, 0, 0
    if t_ < 1: # Down
        x = (final_standing_height - L1 - L2) * t_ + L1 + L2
        y = 0
        q1_1, q2_1, q1_2, q2_2 = q1q2_from_xy(x, y)
    elif t_ < 2: # Up
        x = (L1 + L2 - final_standing_height) * t_ + 2 * final_standing_height - L1 - L2
        y = 0
        q1_1, q2_1, q1_2, q2_2 = q1q2_from_xy(x, y)
    print("Sp 1:", q1_2 * (10 / (2 * math.pi)))
    print("Sp 2:", q2_2 * (10 / (2 * math.pi)))
    # print(x, y)
    t_ += 0.001
    # time.sleep(0.001)

Sp 1: 0.0
Sp 2: 0.0
Sp 1: 0.021460610731046433
Sp 2: -0.042921221462092866
Sp 1: 0.030350116688431072
Sp 2: -0.060700233376862145
Sp 1: 0.03717143138443803
Sp 2: -0.07434286276887606
Sp 1: 0.04292219703681123
Sp 2: -0.08584439407362246
Sp 1: 0.04798883876394557
Sp 2: -0.09597767752789114
Sp 1: 0.05256953732457064
Sp 2: -0.10513907464914128
Sp 1: 0.0567820202643873
Sp 2: -0.11356404052877461
Sp 1: 0.06070299300104021
Sp 2: -0.1214059860020804
Sp 1: 0.06438573489115416
Sp 2: -0.1287714697823083
Sp 1: 0.06786903801357529
Sp 2: -0.13573807602715057
Sp 1: 0.07118218703119582
Sp 2: -0.14236437406239164
Sp 1: 0.07434793304093149
Sp 2: -0.14869586608186297
Sp 1: 0.0773843686583214
Sp 2: -0.1547687373166428
Sp 1: 0.0803061631951581
Sp 2: -0.1606123263903162
Sp 1: 0.08312540617593137
Sp 2: -0.16625081235186273
Sp 1: 0.08585220106685518
Sp 2: -0.17170440213371035
Sp 1: 0.08849509408140299
Sp 2: -0.17699018816280598
Sp 1: 0.09106139082116935
Sp 2: -0.1821227816423387
Sp 1: 0.09355739465414067
Sp 2

In [20]:
import time
import math

time.sleep(1)

# odrv0.axis0.controller.config.control_mode = CONTROL_MODE_POSITION_CONTROL
# odrv0.axis0.controller.config.input_mode = INPUT_MODE_PASSTHROUGH
# odrv0.axis1.controller.config.control_mode = CONTROL_MODE_POSITION_CONTROL
# odrv0.axis1.controller.config.input_mode = INPUT_MODE_PASSTHROUGH

final_standing_height = 300
t_ = 0

def func1(t):
    x = -20 * math.pi * t + 10 * math.pi
    y = 25 * math.cos(0.05 * x)
    return x,y

def func2(t):
    x = 100 * math.pi * t - 150 * math.pi
    y = 0
    return x,y

# Path type 1
while t_ < 1:
    x, y = func1(t_)
    q1_1, q2_1, q1_2, q2_2 = 0, 0, 0, 0
    q1_1, q2_1, q1_2, q2_2 = q1q2_from_xy(x, y)
    print(math.degrees(q1_2), math.degrees(q2_2))
    # odrv0.axis1.controller.input_pos = q1_2
    # odrv0.axis0.controller.input_pos = q2_2
    t_ += 0.01
    # time.sleep(0.01)

time.sleep(2)

# Path type 2
while t_ < 2:
    x, y = func2(t_)
    q1_1, q2_1, q1_2, q2_2 = q1q2_from_xy(x, y)
    # odrv0.axis1.controller.input_pos = q1_2
    # odrv0.axis0.controller.input_pos = q2_2
    t_ += 0.01
    # time.sleep(0.01)

86.72549046613716 -173.45098093227443
88.2510750266446 -173.58001039115985
89.8318492914617 -173.70469174393037
91.46748043514792 -173.82478901173735
93.15729805916226 -173.94007478645952
94.90026055171866 -174.05033271021153
96.69492495017212 -174.1553601326553
98.53942203268771 -174.25497090804018
100.43143846464851 -174.3489982801885
102.36820782129647 -174.4372977898254
104.34651217530042 -174.51975012576744
106.36269565761357 -174.59626383077634
108.41269096577723 -174.66677776566175
110.49205921600986 -174.731263232747
112.59604284345622 -174.7897256631017
114.71963049737036 -174.84220578158613
116.85763211886345 -174.88878017976174
119.00476170088331 -174.92956124843658
121.15572468518663 -174.96469644772196
123.30530660964351 -174.9943669210593
125.44845952075809 -175.01878548853475
127.58038282269885 -175.0381940815527
129.69659562778878 -175.0528607035008
131.79299825999087 -175.06307601774543
133.8659212786691 -175.06914967415022
135.91216116154126 -175.07140648805137
137.92